# Generación de Datos Sintéticos
Este notebook genera datos sintéticos para estudiantes, asignaturas y notas, y los guarda en archivos CSV.

## Generación del archivo `estudiantes.csv`
Este archivo contiene información sobre los estudiantes, incluyendo su nombre, apellido, edad, sexo, ciudad y fecha de registro.

In [41]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import os

csv_files = ['estudiantes.csv', 'asignaturas.csv', 'notas.csv']
for file in csv_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"Deleted existing {file}")

fake = Faker('es_ES')
np.random.seed(42)
random.seed(42)

def should_introduce_error(error_probability=0.1):
    """Return True with the specified probability."""
    return random.random() < error_probability

num_estudiantes = 1000

estudiantes = []
for i in range(1, num_estudiantes + 1):
    estudiante = {
        'id_estudiante': i,
        'nombre': fake.first_name() if not should_introduce_error() else None,
        'apellido': fake.last_name() if not should_introduce_error() else None,
        'edad': random.randint(18, 40) if not should_introduce_error() else 
               random.choice([None, random.randint(-5, 10), random.randint(80, 120)]),
        'sexo': random.choice(['M', 'F']) if not should_introduce_error() else 
               random.choice([None, 'X', '1', '']),
        'ciudad': fake.city() if not should_introduce_error() else None,
        'fecha_registro': fake.date_between(start_date='-3y', end_date='today') if not should_introduce_error() else 
                         random.choice([None, fake.date_between(start_date='-30y', end_date='-20y')])
    }
    estudiantes.append(estudiante)

df_estudiantes = pd.DataFrame(estudiantes)
df_estudiantes.fillna('null', inplace=True)
df_estudiantes.to_csv('estudiantes.csv', index=False)

Deleted existing estudiantes.csv
Deleted existing asignaturas.csv
Deleted existing notas.csv


C:\Users\Deusto\AppData\Local\Temp\ipykernel_8284\2890562609.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'null' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_estudiantes.fillna('null', inplace=True)


## Generación del archivo `asignaturas.csv`
Este archivo contiene información sobre las asignaturas, incluyendo su nombre, departamento y número de créditos.

In [42]:
asignaturas = [
    {'id_asignatura': 1, 'nombre_asignatura': 'Matemáticas', 'departamento': 'Ciencias', 'creditos': 6},
    {'id_asignatura': 2, 'nombre_asignatura': 'Física', 'departamento': 'Ciencias', 'creditos': 5},
    {'id_asignatura': 3, 'nombre_asignatura': 'Historia', 'departamento': 'Humanidades', 'creditos': 4},
    {'id_asignatura': 4, 'nombre_asignatura': 'Lengua Española', 'departamento': 'Humanidades', 'creditos': 5},
    {'id_asignatura': 5, 'nombre_asignatura': 'Informática', 'departamento': 'Ingeniería', 'creditos': 6},
    {'id_asignatura': 6, 'nombre_asignatura': 'Biología', 'departamento': 'Ciencias', 'creditos': 5},
    {'id_asignatura': 7, 'nombre_asignatura': 'Química', 'departamento': 'Ciencias', 'creditos': 5},
    {'id_asignatura': 8, 'nombre_asignatura': 'Economía', 'departamento': 'Ciencias Sociales', 'creditos': 4},
    {'id_asignatura': 9, 'nombre_asignatura': 'Filosofía', 'departamento': 'Humanidades', 'creditos': 4},
    {'id_asignatura': 10, 'nombre_asignatura': 'Inglés', 'departamento': 'Idiomas', 'creditos': 5}
]

# Adding errors to asignaturas data
for asignatura in asignaturas:
    for key in list(asignatura.keys()):
        if key != 'id_asignatura' and should_introduce_error():  # Keep id_asignatura intact
            if key == 'creditos':
                asignatura[key] = random.choice([None, -1, 0, 100])
            else:
                asignatura[key] = None

df_asignaturas = pd.DataFrame(asignaturas)
df_asignaturas.fillna('null', inplace=True)
df_asignaturas.to_csv('asignaturas.csv', index=False)

## Generación del archivo `notas.csv`
Este archivo contiene información sobre las notas de los estudiantes en diferentes asignaturas, incluyendo la convocatoria y la fecha del examen.

In [43]:
notas = []
convocatorias = ['Ordinaria', 'Extraordinaria']

for estudiante in df_estudiantes['id_estudiante']:
    asignaturas_sample = random.sample(df_asignaturas['id_asignatura'].tolist(), k=3)
    for asignatura in asignaturas_sample:
        nota_record = {
            'id_estudiante': estudiante if not should_introduce_error() else random.choice([None, -100, 9999]),
            'id_asignatura': asignatura if not should_introduce_error() else random.choice([None, -5, 999]),
            'nota': round(np.random.normal(loc=6.5, scale=1.5), 2) if not should_introduce_error() else 
                   random.choice([None, -20, 15, 999]),
            'convocatoria': random.choice(convocatorias) if not should_introduce_error() else 
                           random.choice([None, '', 'Desconocida', 123]),
            'fecha_examen': fake.date_between(start_date='-2y', end_date='today') if not should_introduce_error() else
                           random.choice([None, fake.date_between(start_date='-30y', end_date='-20y')])
        }
        notas.append(nota_record)

df_notas = pd.DataFrame(notas)
# Only clip valid notes (not None)
df_notas['nota'] = df_notas['nota'].apply(lambda x: max(0, min(10, x)) if pd.notna(x) and isinstance(x, (int, float)) else x)
df_notas.to_csv('notas.csv', index=False)